In [1]:
import sys
sys.path.append("..") # Ensure the parent directory is in the path

import core.assingment as Models
from core.DataLoader import DataPreprocessor, DataConfig
import core
import numpy as np
from importlib import reload
import keras
import matplotlib.pyplot as plt
MAX_JETS = 6
PLOTS_DIR = f"plots/"
import os
if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)

config = DataConfig(jet_features=["ordered_jet_pt", "ordered_jet_e", "ordered_jet_phi", "ordered_jet_eta", "ordered_jet_b_tag", "dR_l1j", "dR_l2j", "m_l1j", "m_l2j"], 
                                lepton_features=["lep_pt","lep_e", "lep_eta", "lep_phi"],
                                jet_truth_label="ordered_event_jet_truth_idx", 
                                lepton_truth_label="event_lepton_truth_idx", 
                                global_features = ["met_met_NOSYS","met_phi_NOSYS"], 
                                max_leptons=2, 
                                max_jets = MAX_JETS, 
                                non_training_features =["truth_ttbar_mass", "truth_ttbar_pt", "N_jets"], 
                                event_weight="weight_mc_NOSYS")

DataProcessor = DataPreprocessor(config)
DataProcessor.load_data("/data/dust/group/atlas/ttreco/full_training.root", "reco", max_events=1000000)
DataProcessor.normalise_data()
X_train,y_train, X_val, y_val = DataProcessor.split_data(test_size=0.1, random_state=42)

2025-10-02 16:08:02.062427: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-02 16:08:02.063268: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-02 16:08:02.067110: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-02 16:08:02.076597: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759414082.091873  271690 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759414082.09

In [2]:
reload(Models)
reload(core)
TransformerMatcher = Models.FeatureConcatTransformer(config, name="Transformer")

TransformerMatcher.build_model(
    num_heads=8,
    hidden_dim=32,
    num_layers=4,
    dropout_rate=0
)
TransformerMatcher.compile_model(
    loss = core.utils.AssignmentLoss(lambda_excl=5), optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-5), metrics=[core.utils.AssignmentAccuracy()]
)
TransformerMatcher.model.summary()
#TransformerMatcher.load_model("Transformer_Assignment.keras")

2025-10-02 16:08:46.942312: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "FeatureConcatTransformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ global_inputs       │ (None, 1, 2)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_inputs          │ (None, 2, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 2)         │          0 │ global_inputs[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 8)         │          0 │ lep_inputs[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_inputs          │ (None, 6, 9)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 6, 2)      │          0 │ flatten[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_1     │ (None, 6, 8)      │          0 │ flatten_1[0][0]   │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 6, 19)     │          0 │ jet_inputs[0][0], │
│ (Concatenate)       │                   │            │ repeat_vector[0]… │
│                     │                   │            │ repeat_vector_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_embedding (MLP) │ (None, 6, 32)     │      1,902 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_mask            │ (None, 6, 1)      │          0 │ jet_inputs[0][0]  │
│ (GenerateMask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 32)     │      8,544 │ jet_embedding[0]… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 32)     │      8,544 │ self_attention_b… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 32)     │      8,544 │ self_attention_b… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 32)     │      8,544 │ self_attention_b… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_output_embeddi… │ (None, 6, 2)      │        473 │ self_attention_b… │
│ (MLP)               │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_assignment_pro… │ (None, 6, 2)      │          0 │ jet_output_embed… │
│ (TemporalSoftmax)   │                   │            │ jet_mask[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 36,551 (142.78 KB)

 Trainable params: 36,551 (142.78 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
TransformerMatcher.train_model(epochs=30,
                                X_train=X_train,
                                y_train=y_train,
                                sample_weights=core.utils.compute_sample_weights(X_train, y_train),
                                batch_size=512,
                                callbacks = keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, restore_best_weights=True, mode ="max"))

Epoch 1/30
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 192s 132ms/step - assignment_accuracy: 0.4842 - loss: 0.4978 - val_assignment_accuracy: 0.6809 - val_loss: 0.3359
Epoch 2/30
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 200s 142ms/step - assignment_accuracy: 0.6946 - loss: 0.3186 - val_assignment_accuracy: 0.7294 - val_loss: 0.2732
Epoch 3/30
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 181s 129ms/step - assignment_accuracy: 0.7376 - loss: 0.2687 - val_assignment_accuracy: 0.7574 - val_loss: 0.2518
Epoch 4/30
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 183s 130ms/step - assignment_accuracy: 0.7584 - loss: 0.2501 - val_assignment_accuracy: 0.7661 - val_loss: 0.2421
Epoch 5/30
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 187s 133ms/step - assignment_accuracy: 0.7678 - loss: 0.2428 - val_assignment_accuracy: 0.7736 - val_loss: 0.2360
Epoch 6/30
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 177s 126ms/step - assignment_accuracy: 0.7741 - loss: 0.2361 - val_assignment_accuracy: 0.7802 - val_loss: 0.2323
Epoch 7/30
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 181s 129ms/step - assi

In [ ]:
TransformerMatcher.save_model("Transformer_Assignment.keras")

In [ ]:
pred_val = TransformerMatcher.predict_indices(X_val, batch_size=1024)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(y_val[:,:,0].argmax(axis=-1),pred_val[:,:,0].argmax(axis=-1), normalize="true")


In [ ]:
reload(core)
ml_eval = core.assingment.MLEvaluatorBase(TransformerMatcher, X_val, y_val)
importances = ml_eval.compute_permutation_importance(num_repeats=5)

In [ ]:
ml_eval.plot_feature_importance(importances)

In [ ]:
pred_val